Test action space sample

In [1]:
from sb3_contrib import MaskablePPO
import edge_relocation as er


Test masks

In [2]:
max_trajectory_length = 250
min_trajectory_length = 250
initial_load = "variable_load" # low (10-40%), medium(40-60%)), high(60-80%), random (10-80%), variable_load ( different initial load for each episode)

# create environment
erEnv = er.EdgeRelEnv("topoconfig.json", min_trajectory_length, max_trajectory_length, initial_load)
erEnv.action_space.sample()


19

In [ ]:
erEnv.reset()
print("App reqlatency: ", erEnv.mecApp.app_req_latency, "current MEC ID: ", erEnv.mecApp.current_MEC.id, "current cell ID: ", erEnv.mecApp.user_position)
for mec in erEnv.mec_nodes:
    print(mec.id, "cpu util: ", mec.cpu_utilization, "mem util: ", mec.memory_utilization)
print(erEnv.action_masks())

#please note that masks refers to the valid action of selecting 'mec1', mec2, mec3 etc, while the list below lists mecs in wrong order

Test Maskable PPO

In [2]:
import edge_relocation as er

import datetime
max_trajectory_length = 250
min_trajectory_length = 250
initial_load = "variable_load" # low (10-40%), medium(40-60%)), high(60-80%), random (10-80%), variable_load ( different initial load for each episode)


# create environment

erEnv = er.EdgeRelEnv("topoconfig.json", min_trajectory_length, max_trajectory_length, initial_load)
# obs = erEnv.reset()
log_dir = "../logs/PPO/masked/" + datetime.datetime.now().strftime("Test_Mask_NewRewardFunct%Y%m%d-%H%M%S")
model = MaskablePPO("MultiInputPolicy", erEnv, verbose=0, tensorboard_log = log_dir, n_steps=256)
model.learn(total_timesteps=500000)
model.save("masked_model_trajectory250_steps500k_variableload_newReward")
print("episodes number: ", erEnv.episodes_counter)


#Seems that for today, MaskablePPO is checking about the type of Space, and it accepts only spaces of gym, not yet a spaces of gymnasium. Further checks required

All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask - modify and True current MEC
All falses in mask -

5566545+454+5412384049++8049+Test manually single episode

In [ ]:
erEnv = er.EdgeRelEnv("topoconfig.json")
obs = erEnv.reset()
print("trajectory (len) : ", len(erEnv.trajectory) ,erEnv.trajectory)
print("App reqlatency: ", erEnv.mecApp.app_req_latency, "current MEC ID: ", erEnv.mecApp.current_MEC.id, "current cell ID: ", erEnv.mecApp.user_position)
for i in range (len(erEnv.trajectory)):
    print("masks:", erEnv.action_masks())
    print("current_cell:", erEnv.trajectory[i])
    obs, rewards, dones, info = erEnv.step(3)
    print(obs)
    print(rewards)
    print(dones)